In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
import keras
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

2024-08-23 06:32:32.257538: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-23 06:32:32.268386: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-23 06:32:32.268420: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-23 06:32:32.275764: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-23 06:32:32.800505: W tensorflow/compiler/tf

In [2]:
# GPU 장치 목록 확인
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow는 GPU를 사용할 수 있습니다.")
    for gpu in gpus:
        print(f"사용 가능한 GPU: {gpu}")

    # gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
    # print(gpus)
    # if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)
else:
    print("TensorFlow는 GPU를 사용할 수 없습니다.")

TensorFlow는 GPU를 사용할 수 있습니다.
사용 가능한 GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-08-23 06:32:36.553312: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-23 06:32:36.569648: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-23 06:32:36.569689: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
# MNIST 데이터셋 로드
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

In [4]:
# 데이터 전처리
x_train = x_train.reshape(-1, 32, 32, 3).astype('float32') / 255.0
x_test = x_test.reshape(-1, 32, 32, 3).astype('float32') / 255.0
y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

In [ ]:
target_size = (224, 224)
batch_size = 64

def process_batch(images, target_size):
    resized_images = []
    for img in images:
        resized_image = tf.image.resize(img, target_size)
        resized_images.append(resized_image)
    return tf.stack(resized_images)

# 메모리에 한 번에 올리지 않고 배치 단위로 처리
def resize_in_batches(images, target_size, batch_size):
    num_batches = len(images) // batch_size
    resized_images = []
    for i in range(num_batches):
        batch = images[i * batch_size:(i + 1) * batch_size]
        resized_images.extend(process_batch(batch, target_size))
    return np.array(resized_images)

In [5]:
resized_x_train = tf.image.resize(x_train, (224, 224))
resized_x_test = tf.image.resize(x_test, (224, 224))

2024-08-23 06:32:46.453576: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-23 06:32:46.453650: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-23 06:32:46.453671: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-23 06:32:46.583477: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-23 06:32:46.583529: I external/local_xla/xla/stream_executor

ResourceExhaustedError: {{function_node __wrapped__ResizeBilinear_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[50000,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ResizeBilinear] name: 